fork元: https://github.com/richarddwang/electra_pytorch

miyamonzがこれを読解した上で、コードの整理とコメントの追加などをしたものです。

以下に依存をしています。
- pytorch, fastai
- ELECTRAのモデル定義に関しては、huggingface/transformers
- 学習データはhuggingface/datasets

モデルのレイヤーの定義をメジャーなライブラリに頼ることで、全体のコード量が減っていて把握しやすいかと思います。

行数が多く読解が難しいのは以下あたりかとおもいます

- 前処理の部分（ELECTRADataProcessor）
- 事前学習タスクの定義部分（ELECTRAModel, ELECTRALoss）


元のPretrain.ipynbからの変更ポイント

- star importをやめた
- _utilsのフォルダのうち、pretrianでしか使われていないコードをpretrain/_utilsに移動
- ノートブック内で記述された長い処理を、別ファイルに切り出してpretrain/*.pyに移動

In [ ]:
import os
import random
from pathlib import Path
from datetime import datetime, timezone, timedelta
import numpy as np
import torch

from transformers import ElectraConfig, ElectraTokenizerFast, ElectraForMaskedLM, ElectraForPreTraining

# 1. Configuraton

もともとは_utilsにあったが、シンプルなクラスなのでnotebook直書きに移動

ただ単に`c["hoge"]`を`c.hoge`で書けるようにするだけのクラス

In [ ]:
class MyConfig(dict):
    def __getattr__(self, name):
        return self[name]
    def __setattr__(self, name, value):
        self[name] = value

In [ ]:
c = MyConfig({
    'device': 'cuda:0',
    'base_run_name': 'vanilla',  # run_name = {base_run_name}_{seed}
    'seed': 11081,  # 11081 36 1188 76 1 4 4649 7 # None/False to randomly choose seed from [0,999999]

    'adam_bias_correction': False,
    'schedule': 'original_linear',
    'sampling': 'fp32_gumbel',
    'electra_mask_style': True,
    'gen_smooth_label': False,
    'disc_smooth_label': False,

    'size': 'small',
#     'datas': ['openwebtext'],
    'datas': ['wikipedia'],
    'logger': None, #"wandb",
    'num_workers': 3,
})


""" Vanilla ELECTRA settings
'adam_bias_correction': False,
'schedule': 'original_linear',
'sampling': 'fp32_gumbel',
'electra_mask_style': True,
'gen_smooth_label': False,
'disc_smooth_label': False,
'size': 'small',
'datas': ['openwebtext'],
"""

以下のセルでは設定の確認や、設定値から別の設定を定めたい処理などを行っている

ほとんど元と同じだが、不要なフォルダ作成処理などは消した

In [ ]:
# Check and Default
assert c.sampling in ['fp32_gumbel', 'fp16_gumbel', 'multinomial']
assert c.schedule in ['original_linear', 'separate_linear', 'one_cycle', 'adjusted_one_cycle']
for data in c.datas:
    assert data in ['wikipedia', 'bookcorpus', 'openwebtext']
assert c.logger in ['wandb', 'neptune', None, False]

if not c.base_run_name:
    c.base_run_name = str(datetime.now(timezone(timedelta(hours=+8))))[6:-13].replace(' ','').replace(':','').replace('-','')
if not c.seed:
    c.seed = random.randint(0, 999999)

c.run_name = f'{c.base_run_name}_{c.seed}'

if c.gen_smooth_label is True:
    c.gen_smooth_label = 0.1
if c.disc_smooth_label is True:
    c.disc_smooth_label = 0.1

# Setting of different sizes
i = ['small', 'base', 'large'].index(c.size)
c.mask_prob = [0.15, 0.15, 0.25][i]
c.lr = [5e-4, 2e-4, 2e-4][i]
c.bs = [128, 256, 2048][i]
c.steps = [10**6, 766*1000, 400*1000][i]
c.max_length = [128, 512, 512][i]
generator_size_divisor = [4, 3, 4][i]

disc_config = ElectraConfig.from_pretrained(f'google/electra-{c.size}-discriminator')
gen_config = ElectraConfig.from_pretrained(f'google/electra-{c.size}-generator')
# note that public electra-small model is actually small++ and don't scale down generator size 
gen_config.hidden_size = int(disc_config.hidden_size/generator_size_divisor)
gen_config.num_attention_heads = disc_config.num_attention_heads//generator_size_divisor
gen_config.intermediate_size = disc_config.intermediate_size//generator_size_divisor
hf_tokenizer = ElectraTokenizerFast.from_pretrained(f"google/electra-{c.size}-generator")

# Print info
print(f"process id: {os.getpid()}")
print(c)

In [ ]:
# Path to data
Path('./checkpoints/pretrain').mkdir(exist_ok=True, parents=True)

# 1. Load Data

事前学習に必要なテキストデータの用意

huggingaface/datasetsを利用している

元のノートブックではopenwebtextなどもダウンロードするようになっていたが、私は今後日本語での事前学習をしたいので消した

ここは後に別ファイルに切り出す可能性が高い

In [ ]:
import datasets
def download_dataset():
    # download english
    # wiki = datasets.load_dataset('wikipedia', '20200501.en')['train']
    
    # download japanese
    wiki = datasets.load_dataset(
        './pretrain/wikipedia.py',
        beam_runner='DirectRunner',
        language='ja',
        date='20210120')['train']
    return wiki

wiki = download_dataset()

以下ではpreprocessを行っている

具体的な処理は、ELECTRADataProcessorというクラスで定義されている

これは、元のgoogleの実装と同じことをするようにpytorchで書き直されたもの

ただし、このクラスは、実装者であるricharddwang氏が書いたhugdatafastというライブラリに依存していた

huggingface/datasetsのデータセットとfastaiをよしなにつなぐためのものらしいが、内部で何をしてるのか分かりにくいので、このライブラリへの依存を外した


今回のユースケースに置いて、hugdatafastのコードがやっていることは殆どなかったからだ。  
（このことを確認してもらうためには、実際にコードを読んで確認してもらうしか無い。とても面倒だった

結果として、そのライブラリが持っていたHF_Dataset, MySortedDLというクラスを、不要なものを削除してepretrain/_utils/{hf_dataset.py, mysorteddl.py}にコピーした。

この書き換えが、ケアレスミス等が無くうまく言っているか保証がないしちょっと不安なのだが、事前学習はちゃんと動いたので多分大丈夫

In [ ]:
from pathlib import Path
from functools import partial
from pretrain._utils.electra_dataprocessor import ELECTRADataProcessor
data_dir = Path('./data')
def preprocess(sources, c, hf_tokenizer, num_proc):
    dsets = []
    ELECTRAProcessor = partial(
        ELECTRADataProcessor, hf_tokenizer=hf_tokenizer, max_length=c.max_length)
    
    for name, ds in sources.items():
        cache_dir = data_dir / "preprocess" / f"{name}_{len(ds)}_{c.max_length}"
        cache_dir.mkdir(parents=True, exist_ok=True)
        path = cache_dir / f"electra.arrow"
        
        cache_file_name = str(path.resolve())
        mapped = ELECTRAProcessor(ds).map(cache_file_name=cache_file_name, num_proc=num_proc)
        dsets.append(mapped)

    assert len(dsets) == len(sources)

    train_dset = datasets.concatenate_datasets(dsets)
    return train_dset

結構時間がかかるので注意

In [ ]:
%%time
# it took about 25min. less num_proc will increase time.
# after cache is created, it takes about 40s
sources = {
    'wiki': wiki,
}
train_dset = preprocess(sources, c, hf_tokenizer, num_proc=16)

ここらへんは元はricharddwang/hugdatafastのコードがやっていたものなのだが、解体した結果こうなった

In [ ]:
from pretrain.get_dataloaders import get_dataloader
dl = get_dataloader(c, hf_tokenizer, train_dset)

from fastai.text.all import DataLoaders
dls = DataLoaders(dl, path='.')

In [ ]:
len(dls.train)

# 2. Masked language model objective

## 2.1 MLM objective callback

モデルにデータを渡す前に、マスクをかけたりするところ  
つまりELECTRAの事前学習タスクについての知識ないと難しいかも

In [ ]:
from pretrain.masked_lm_cb import MaskedLMCallback

In [ ]:
mlm_cb = MaskedLMCallback(mask_tok_id=hf_tokenizer.mask_token_id,
                          special_tok_ids=hf_tokenizer.all_special_ids,
                          vocab_size=hf_tokenizer.vocab_size,
                          mlm_probability=c.mask_prob,
                          replace_prob=0.0 if c.electra_mask_style else 0.1,
                          original_prob=0.15 if c.electra_mask_style else 0.1,
                          for_electra=True)

# 3. ELECTRA (replaced token detection objective)
see details in paper [ELECTRA: Pre-training Text Encoders as Discriminators Rather Than Generators](https://arxiv.org/abs/2003.10555)

In [ ]:
from pretrain.models import ELECTRAModel, ELECTRALoss

# 5. Train

In [ ]:
# Seed & PyTorch benchmark
torch.backends.cudnn.benchmark = True
dls[0].rng = random.Random(c.seed) # for fastai dataloader
random.seed(c.seed)
np.random.seed(c.seed)
torch.manual_seed(c.seed)

modelの用意をしている

In [ ]:
# Generator and Discriminator
generator = ElectraForMaskedLM(gen_config)
discriminator = ElectraForPreTraining(disc_config)
discriminator.electra.embeddings = generator.electra.embeddings
generator.generator_lm_head.weight = generator.electra.embeddings.word_embeddings.weight

# ELECTRA training loop
electra_model = ELECTRAModel(generator, discriminator, hf_tokenizer, sampling=c.sampling)
electra_loss_func = ELECTRALoss(gen_label_smooth=c.gen_smooth_label, disc_label_smooth=c.disc_smooth_label)

optimizerとschedulerは別のファイルに分けた

中身については参考元のREADMEのAdvanced detailsをよく読むとよい

TODO: optimizerとschedulerについてもう少し詳しく書く

In [ ]:
from pretrain.optim import get_optim
opt_func = get_optim(c)

In [ ]:
from pretrain.scheduler import get_scheduler
lr_shedule = get_scheduler(c)

以下から、学習の実行になる

fastaiのLearnerとか、ここからfastaiで学習する際の呼び出し方の話になる

callbackとして、以下を渡している
- MaskedLMCallback
- RunSteps

RunStepsにて、学習全体の完了の判定と、checkpointの保存などを行っている

In [ ]:
from fastai.text.all import Learner
from pretrain._utils.run_steps import RunSteps
# Learner
dls.to(torch.device(c.device))
learn = Learner(dls, electra_model,
                loss_func=electra_loss_func,
                opt_func=opt_func,
                path='./checkpoints',
                model_dir='pretrain',
                cbs=[mlm_cb,
                    RunSteps(c.steps, [0.0625, 0.125, 0.25, 0.5, 1.0], c.run_name+"_{percent}"),
                    ],
                )

In [ ]:
if c.logger == 'wandb':
  import wandb
  from fastai.callback.wandb import  WandbCallback
  wandb.init(name=c.run_name, project='electra_pretrain', config={**c})
  learn.add_cb(WandbCallback(log_preds=False, log_model=False))

In [ ]:
from _utils.gradient_clipping import GradientClipping
# Mixed precison and Gradient clip
learn.to_fp16(init_scale=2.**11)
learn.add_cb(GradientClipping(1.))

学習全体の停止は、既に述べた通りRunStepsというコールバックで制御される。

なのでfitにわたすepoch数は適当にでかい数字を入れている


なんでこうなっているかは分からないが、RunStepsで制御してる内容が、恐らく素のfastaiのままだと実現できなかったからだろうと思われる。

In [ ]:
# pretraining will stop by c.steps and it's controlled by RunSteps callback.
# 9999 is no meaning.
learn.fit(9999, cbs=[lr_shedule])